In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import math

In [2]:
data = pd.read_csv('./data.csv',index_col = 0)

In [3]:
data.shape

(30697, 27)

In [4]:
data.head()

,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,remaining_sec,distance_of_shot,is_goal,...,lat/lng,type_of_shot,type_of_combined_shot,match_id,team_id,remaining_min.1,power_of_shot.1,knockout_match.1,remaining_sec.1,distance_of_shot.1
0,10.0,167.0,72.0,10.0,1.0,0.0,2000-01,27.0,38.0,NaN,...,"45.539131, -122.651648",shot - 30,NaN,20000012,1610612747,10.00,1.0,50.608,54.2000,38.0
1,12.0,-157.0,0.0,10.0,1.0,0.0,2000-01,22.0,35.0,0.0,...,"45.539131, -122.651648",shot - 45,NaN,20000012,1610612747,10.00,1.0,28.800,22.0000,35.0
2,35.0,-101.0,135.0,7.0,1.0,0.0,2000-01,45.0,36.0,1.0,...,"45.539131, -122.651648",shot - 25,NaN,20000012,1610612747,92.64,1.0,0.000,63.7216,54.4
3,43.0,138.0,175.0,6.0,1.0,0.0,2000-01,52.0,42.0,0.0,...,"45.539131, -122.651648",NaN,shot - 3,20000012,1610612747,NaN,1.0,122.608,52.0000,42.0
4,155.0,0.0,0.0,NaN,2.0,0.0,2000-01,19.0,20.0,1.0,...,"45.539131, -122.651648",NaN,shot - 1,20000012,1610612747,42.64,2.0,0.000,19.0000,20.0


In [5]:
data.describe(include = ['object'])

,game_season,area_of_shot,shot_basics,range_of_shot,team_name,date_of_game,home/away,lat/lng,type_of_shot,type_of_combined_shot
count,24835,29195,29122,29133,29162,29147,29200,29132,15417,15280
unique,20,6,7,5,1,1558,74,38,57,6
top,2005-06,Center(C),Mid Range,Less Than 8 ft.,Manchester United,2016-04-13,MANU @ SAS,"42.982923, -71.446094",shot - 39,shot - 3
freq,1911,12761,11955,8933,29162,49,971,14171,1445,11685


# HOME/AWAY

In [6]:
temp = data
column = ['home/away']
temp.loc[:,column] = temp.loc[:,column].ffill()

In [7]:
def replace_(x):
    
    if type(x) == str :
        if x.find('@') != -1 :
            return 1
        else :
            return 0
    else :
        return -1

In [8]:
temp['home_index'] = temp['home/away'].apply(replace_)
temp['home/away'] = temp['home/away'].str.replace(' @ ','')
temp['home/away'] = temp['home/away'].str.replace(' vs. ','')

In [9]:
temp.describe(include = ['object'])

,game_season,area_of_shot,shot_basics,range_of_shot,team_name,date_of_game,home/away,lat/lng,type_of_shot,type_of_combined_shot
count,24835,29195,29122,29133,29162,29147,30697,29132,15417,15280
unique,20,6,7,5,1,1558,38,38,57,6
top,2005-06,Center(C),Mid Range,Less Than 8 ft.,Manchester United,2016-04-13,MANUSAS,"42.982923, -71.446094",shot - 39,shot - 3
freq,1911,12761,11955,8933,29162,49,1955,14171,1445,11685


temp.to_csv(r'./hello.csv')

# DELETE UNECESSARY

In [10]:
del temp['match_event_id']

In [11]:
del temp['match_id']

In [12]:
del temp['team_id']

In [13]:
del temp['team_name']

In [14]:
del temp['date_of_game']

# POWER_OF_SHOT

In [15]:
def replace_power(x):
    
    if math.isnan(x['power_of_shot']) :
        
        if x['power_of_shot.1'] in [1,2,3,4,5,6,7] :
            return x['power_of_shot.1']
        else :
            return 3
        
    else:
        return x['power_of_shot']
    

In [16]:
temp['power_of_shot_new'] = temp.apply(replace_power,axis=1)
del temp['power_of_shot']
del temp['power_of_shot.1']

# REMAINING_MIN

In [17]:
def replace_min(x):
    
    if math.isnan(x['remaining_min']) :
        
        if x['remaining_min.1'] in [0,1,2,3,4,5,6,7,8,9,10,11] :
            return x['remaining_min.1']
        else :
            return 0
        
    else:
        return x['remaining_min']

In [18]:
temp['remaining_min_new'] = temp.apply(replace_min,axis=1)
del temp['remaining_min']
del temp['remaining_min.1']

# REMAINING_SEC

In [19]:
def replace_sec(x):
    
    if math.isnan(x['remaining_sec']) :
        
        if x['remaining_sec.1'] in np.arange(60) :
            return x['remaining_sec.1']
        else :
            return 30
        
    else:
        return x['remaining_sec']

In [20]:
temp['remaining_sec_new'] = temp.apply(replace_sec,axis=1)
del temp['remaining_sec']
del temp['remaining_sec.1']

# CREATE_REMAINING TIME

In [21]:
temp['remaining_time_new'] = temp['remaining_min_new']*60 + temp['remaining_sec_new']
del temp['remaining_sec_new']
del temp['remaining_min_new']

# DISTANCE OF SHOT

In [22]:
def replace_distance(x):
    
    if math.isnan(x['distance_of_shot']) :
        
        if x['distance_of_shot.1'] in np.arange(20,100) :
            return x['distance_of_shot.1']
        elif x['range_of_shot'] == "Less Than 8ft." :
            return 20
        elif x['range_of_shot'] == "Back Court Shot" :
            return 73
        elif x['range_of_shot'] == "8-16 ft." :
            return 32
        elif x['range_of_shot'] == "24+ ft." :
            return 46
        elif x['range_of_shot'] == "16-24 ft." :
            return 39
        else :
            return 20
    else:
        return x['distance_of_shot']

In [23]:
temp['distance_of_shot_new'] = temp.apply(replace_distance,axis=1)
del temp['distance_of_shot']
del temp['distance_of_shot.1']

# KNOCKOUT_MATCH

In [24]:
column = ['knockout_match']
temp.loc[:,column] = temp.loc[:,column].ffill()
del temp['knockout_match.1']

# LOCATION_X

In [25]:
def replace_x(t):
    
    if math.isnan(t['location_x']) :
        
        if not(math.isnan(t['location_y'])):
            
            val = t['location_y']
            
            df = returndata(val)
            
            if (type(t['area_of_shot']) == str) :
                
                area = t['area_of_shot']
                val_1 = df[df['area_of_shot'] == area]['location_x'].abs().mean()
                
                if math.isnan(val_1) :
                    val_2 =  temp[(temp['area_of_shot'] == area)& (temp['location_x'].notna()) ]['location_x'].abs().mean()
                    return round(val_2)
                else :
                    return round(val_1)
            
            else :
                
                val_1 = df['location_x'].abs().mean()
                if math.isnan(val_1) :
                    val_2 =  temp[temp['location_x'].notna()]['location_x'].abs().mean()
                    return round(val_2)
                else :
                    return round(val_1)
            
        else :
            
            df = returndata1()
            if (type(t['area_of_shot']) == str) :
                
                area = t['area_of_shot']
                val_1 = df[df['area_of_shot'] == area]['location_x'].abs().mean()
                if math.isnan(val_1) :
                    val_2 = temp[(temp['area_of_shot'] == area)& (temp['location_x'].notna()) ]['location_x'].abs().mean()
                    return round(val_2)
                else :
                    return round(val_1)
            
            else :
                
                val_1 = df['location_x'].abs().mean()
                if math.isnan(val_1) :
                    val_2 = temp[temp['location_x'].notna()]['location_x'].abs().mean()
                    return round(val_2)
                else :
                    return round(val_1)
             
            
    else:
        return t['location_x']
    

def returndata(val):
    
    df = temp[(abs(temp['location_y'] - val) <= 5) & (temp['location_x'].notna())]
    return df

def returndata1():
    
    df = temp[temp['location_x'].notna()]
    return df


In [26]:
temp['location_x_new'] = temp.apply(replace_x,axis=1)
del temp['location_x']

# LOCATION_Y

In [27]:
def replace_y(t):
    
    if math.isnan(t['location_y']) :
        
        if not(math.isnan(t['location_x_new'])):
            
            val = t['location_x_new']

            df = returndata(val)
            
            if (type(t['area_of_shot']) == str) :
                
                area = t['area_of_shot']
                val_1 = df[df['area_of_shot'] == area]['location_y'].abs().mean()
                
                if math.isnan(val_1) :
                    return round(temp[(temp['area_of_shot'] == area)& (temp['location_y'].notna()) ]['location_y'].abs().mean())
                else :
                    return round(val_1)
            
            else :
                
                val_1 = df['location_y'].abs().mean()
                if math.isnan(val_1) :
                    return round(temp[temp['location_y'].notna()]['location_y'].abs().mean())
                else :
                    return round(val_1)
            
        else :
            
            df = returndata1()
            if (type(t['area_of_shot']) == str) :
                
                area = t['area_of_shot']
                val_1 = df[df['area_of_shot'] == area]['location_y'].abs().mean()
                if math.isnan(val_1) :
                    return round(temp[(temp['area_of_shot'] == area)& (temp['location_y'].notna()) ]['location_y'].abs().mean())
                else :
                    return round(val_1)
            
            else :
                
                val_1 = df['location_y'].abs().mean()
                if math.isnan(val_1) :
                    return round(temp[temp['location_y'].notna()]['location_y'].abs().mean())
                else :
                    return round(val_1)
             
            
    else:
        return t['location_y']
    

def returndata(val):
    
    df = temp[(abs(temp['location_x_new'] - val) <= 5) & (temp['location_y'].notna())]
    return df

def returndata1():
    
    df = temp[temp['location_y'].notna()]
    return df


In [28]:
temp['location_y_new'] = temp.apply(replace_y,axis=1)
del temp['location_y']

In [29]:
temp.head()

,knockout_match,game_season,is_goal,area_of_shot,shot_basics,range_of_shot,home/away,shot_id_number,lat/lng,type_of_shot,type_of_combined_shot,home_index,power_of_shot_new,remaining_time_new,distance_of_shot_new,location_x_new,location_y_new
0,0.0,2000-01,NaN,Right Side(R),Mid Range,16-24 ft.,MANUPOR,1.0,"45.539131, -122.651648",shot - 30,NaN,1,1.0,627.0,38.0,167.0,72.0
1,0.0,2000-01,0.0,Left Side(L),Mid Range,8-16 ft.,MANUPOR,2.0,"45.539131, -122.651648",shot - 45,NaN,1,1.0,622.0,35.0,-157.0,0.0
2,0.0,2000-01,1.0,Left Side Center(LC),Mid Range,16-24 ft.,MANUPOR,3.0,"45.539131, -122.651648",shot - 25,NaN,1,1.0,465.0,36.0,-101.0,135.0
3,0.0,2000-01,0.0,Right Side Center(RC),Mid Range,16-24 ft.,MANUPOR,4.0,"45.539131, -122.651648",NaN,shot - 3,1,1.0,412.0,42.0,138.0,175.0
4,0.0,2000-01,1.0,Center(C),Goal Area,Less Than 8 ft.,MANUPOR,5.0,"45.539131, -122.651648",NaN,shot - 1,1,2.0,19.0,20.0,0.0,0.0


# DELETE REMAINING

In [30]:
del temp['area_of_shot']
del temp['range_of_shot']
del temp['shot_id_number']

In [31]:
temp.head()

,knockout_match,game_season,is_goal,shot_basics,home/away,lat/lng,type_of_shot,type_of_combined_shot,home_index,power_of_shot_new,remaining_time_new,distance_of_shot_new,location_x_new,location_y_new
0,0.0,2000-01,NaN,Mid Range,MANUPOR,"45.539131, -122.651648",shot - 30,NaN,1,1.0,627.0,38.0,167.0,72.0
1,0.0,2000-01,0.0,Mid Range,MANUPOR,"45.539131, -122.651648",shot - 45,NaN,1,1.0,622.0,35.0,-157.0,0.0
2,0.0,2000-01,1.0,Mid Range,MANUPOR,"45.539131, -122.651648",shot - 25,NaN,1,1.0,465.0,36.0,-101.0,135.0
3,0.0,2000-01,0.0,Mid Range,MANUPOR,"45.539131, -122.651648",NaN,shot - 3,1,1.0,412.0,42.0,138.0,175.0
4,0.0,2000-01,1.0,Goal Area,MANUPOR,"45.539131, -122.651648",NaN,shot - 1,1,2.0,19.0,20.0,0.0,0.0


# SHOT_BASICS

temp_shot_basics = pd.get_dummies(temp['shot_basics'])
temp = pd.concat([temp, temp_shot_basics], axis=1)
del temp['shot_basics']

In [32]:
del temp['shot_basics']

# TYPE OF SHOT

In [33]:
def replace_shot(x):
    
    if type(x['type_of_shot']) == str :
       
        if x['type_of_shot'] in ['shot - 32','shot - 4','shot - 44','shot - 12','shot - 17','shot - 52','shot - 56','shot - 15','shot - 38','shot - 39','shot - 36'] :
            return  "type" + x['type_of_shot']
        else :
            return "Other"
    else:
        return x['type_of_shot']

In [34]:
temp['type_of_shot_new'] = temp.apply(replace_shot,axis=1)
del temp['type_of_shot']

In [35]:
temp_type_shot = pd.get_dummies(temp['type_of_shot_new'])
temp = pd.concat([temp, temp_type_shot], axis=1)
del temp['type_of_shot_new']

del temp['type_of_shot_new']

# COMBINED SHOT

In [36]:
temp_type_shot = pd.get_dummies(temp['type_of_combined_shot'])
temp = pd.concat([temp, temp_type_shot], axis=1)
del temp['type_of_combined_shot']

del temp['type_of_combined_shot']

# HOME/AWAY

In [37]:
temp_home = pd.get_dummies(temp['home/away'])
temp = pd.concat([temp, temp_home], axis=1)
del temp['home/away']

del temp['home/away']

temp.to_csv(r'./hello.csv')

In [38]:
del temp['lat/lng']
del temp['game_season']

In [39]:
temp.head()

,knockout_match,is_goal,home_index,power_of_shot_new,remaining_time_new,distance_of_shot_new,location_x_new,location_y_new,Other,typeshot - 12,...,MANUPOR,MANUSAC,MANUSAN,MANUSAS,MANUSEA,MANUTOR,MANUUTA,MANUUTH,MANUVAN,MANUWAS
0,0.0,NaN,1,1.0,627.0,38.0,167.0,72.0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,0.0,0.0,1,1.0,622.0,35.0,-157.0,0.0,1,0,...,1,0,0,0,0,0,0,0,0,0
2,0.0,1.0,1,1.0,465.0,36.0,-101.0,135.0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,0.0,0.0,1,1.0,412.0,42.0,138.0,175.0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0.0,1.0,1,2.0,19.0,20.0,0.0,0.0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [40]:
dataset_train = temp.loc[temp['is_goal'].notnull()]

In [41]:
dataset_train.shape

(24429, 64)

In [42]:
dataset_test = temp.loc[temp['is_goal'].isnull()]

In [43]:
dataset_test.shape

(6268, 64)

In [44]:
dataset_train.head()

,knockout_match,is_goal,home_index,power_of_shot_new,remaining_time_new,distance_of_shot_new,location_x_new,location_y_new,Other,typeshot - 12,...,MANUPOR,MANUSAC,MANUSAN,MANUSAS,MANUSEA,MANUTOR,MANUUTA,MANUUTH,MANUVAN,MANUWAS
1,0.0,0.0,1,1.0,622.0,35.0,-157.0,0.0,1,0,...,1,0,0,0,0,0,0,0,0,0
2,0.0,1.0,1,1.0,465.0,36.0,-101.0,135.0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,0.0,0.0,1,1.0,412.0,42.0,138.0,175.0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0.0,1.0,1,2.0,19.0,20.0,0.0,0.0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,0.0,0.0,1,3.0,572.0,34.0,-145.0,-11.0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [45]:
index_test = dataset_test.index

In [46]:
y_train = np.asarray(dataset_train['is_goal'])

In [47]:
del dataset_train['is_goal']

In [48]:
dataset_train = dataset_train.fillna(0)

In [49]:
X_train = np.asarray(dataset_train)

In [50]:
X_train.shape

(24429, 63)

In [51]:
del dataset_test['is_goal']

In [52]:
dataset_test = dataset_test.fillna(0)

In [53]:
X_test = np.asarray(dataset_test)

X_train[:,3] = X_train[:,3]/(np.max(X_train[:,3]) - np.min(X_train[:,3]))
X_train[:,4] = X_train[:,4]/(np.max(X_train[:,4]) - np.min(X_train[:,4]))
X_train[:,5] = X_train[:,5]/(np.max(X_train[:,5]) - np.min(X_train[:,5]))
X_train[:,6] = X_train[:,6]/(np.max(X_train[:,6]) - np.min(X_train[:,6]))

X_test[:,3] = X_test[:,3]/(np.max(X_test[:,3]) - np.min(X_test[:,3]))
X_test[:,4] = X_test[:,4]/(np.max(X_test[:,4]) - np.min(X_test[:,4]))
X_test[:,5] = X_test[:,5]/(np.max(X_test[:,5]) - np.min(X_test[:,5]))
X_test[:,6] = X_test[:,6]/(np.max(X_test[:,6]) - np.min(X_test[:,6]))

In [ ]:
clf = LogisticRegression(random_state = 0).fit(X_train,y_train)

In [ ]:
y_train_predict = clf.predict_proba(X_train)

In [ ]:
mean_absolute_error(y_train,y_train_predict[:,1])

In [ ]:
clf.score(X_train,y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=900, max_features='sqrt' ,max_depth=15,min_samples_split = 5,min_samples_leaf = 4,bootstrap=True,random_state=42)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
y_train_predict = rf.predict_proba(X_train)

In [ ]:
mean_absolute_error(y_train,y_train_predict[:,1])

rf = RandomForestClassifier(n_estimators=200, max_features=10 ,max_depth=15,random_state=0)

In [ ]:
feature_importances = pd.DataFrame(rf.feature_importances_,index = dataset_train.columns,columns=['importance']).sort_values('importance',ascending=False)

In [ ]:
print(feature_importances)

In [ ]:
feature_importances.to_csv(r'./importance.csv')

In [ ]:
from pprint import pprint

In [ ]:
pprint(rf.get_params())

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 50, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
dataset_train.head()

In [ ]:
pprint(xgb_model.get_params())

temp.to_csv(r'./hello.csv')

In [54]:
from xgboost import XGBClassifier

In [59]:
xgb_model = XGBClassifier(objective="binary:logistic", random_state=42,max_depth = 3 ,n_estimators = 100,min_child_weight = 2)
xgb_model.fit(X_train, y_train)

y_train_predict = xgb_model.predict_proba(X_train)

In [60]:
mean_absolute_error(y_train,y_train_predict[:,1])

0.4496575382418546

from sklearn.neural_network import MLPClassifier

 clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(64,32,16,8, 2), random_state=1)

clf.fit(X_train, y_train)  

y_train_predict = clf.predict_proba(X_train)

mean_absolute_error(y_train,y_train_predict[:,1])

In [57]:
# XGBoost on Otto dataset, Tune n_estimators
from pandas import read_csv
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot

# grid search
model = XGBClassifier()
n_estimators = range(50,500 , 50)
param_grid = dict(n_estimators=n_estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_mean_absolute_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))
# plot
pyplot.errorbar(n_estimators, means, yerr=stds)
pyplot.title("XGBoost n_estimators vs Log Loss")
pyplot.xlabel('n_estimators')
pyplot.ylabel('Log Loss')
pyplot.savefig('n_estimators.png')

Best: -0.364362 using {'n_estimators': 100}
-0.366450 (0.005431) with: {'n_estimators': 50}
-0.364362 (0.006091) with: {'n_estimators': 100}
-0.364362 (0.006116) with: {'n_estimators': 150}
-0.364812 (0.006637) with: {'n_estimators': 200}
-0.364690 (0.006476) with: {'n_estimators': 250}
-0.366204 (0.007047) with: {'n_estimators': 300}
-0.367023 (0.006763) with: {'n_estimators': 350}
-0.367391 (0.006944) with: {'n_estimators': 400}
-0.368046 (0.006964) with: {'n_estimators': 450}


In [58]:
matplotlib.use('Agg')

# grid search
model = XGBClassifier()
max_depth = range(1, 21, 2)
print(max_depth)
param_grid = dict(max_depth=max_depth)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_mean_absolute_error", n_jobs=-1, cv=kfold, verbose=1)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))
# plot
pyplot.errorbar(max_depth, means, yerr=stds)
pyplot.title("XGBoost max_depth vs Log Loss")
pyplot.xlabel('max_depth')
pyplot.ylabel('Log Loss')
pyplot.savefig('max_depth.png')

range(1, 21, 2)
Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  6.4min finished


Best: -0.364362 using {'max_depth': 3}
-0.373286 (0.003987) with: {'max_depth': 1}
-0.364362 (0.006091) with: {'max_depth': 3}
-0.365713 (0.005037) with: {'max_depth': 5}
-0.370339 (0.009073) with: {'max_depth': 7}
-0.373572 (0.009200) with: {'max_depth': 9}
-0.382783 (0.008828) with: {'max_depth': 11}
-0.389332 (0.007595) with: {'max_depth': 13}
-0.388514 (0.009825) with: {'max_depth': 15}
-0.394613 (0.010524) with: {'max_depth': 17}
-0.400303 (0.007783) with: {'max_depth': 19}


In [61]:
y_test_predict =  xgb_model.predict_proba(X_test)

In [62]:
y_test_predict = y_test_predict[:,1]
y_test_predict = np.around(y_test_predict,decimals=1)

In [63]:
y_test_predict

array([0.4, 0.6, 0.5, ..., 0.7, 0.4, 0.7], dtype=float32)

In [64]:
index_test = dataset_test.index
index_test = index_test + 1

In [65]:
mysubmission = pd.DataFrame( { 'shot_id_number': index_test, 'is_goal' : y_test_predict }, index = index_test )
mysubmission.columns = [ 'shot_id_number', 'is_goal' ]
mysubmission.head()

,shot_id_number,is_goal
1,1,0.4
8,8,0.6
17,17,0.5
20,20,0.7
22,22,0.4


In [66]:
mysubmission.to_csv(f'Saurabh_Jain_270299_prediction_17.csv', index = False )